In [2]:
import imp

In [3]:
sim.agents

NameError: name 'sim' is not defined

In [4]:
''' 1.10 Adding non_habitual agents'''
    parameter = 'adding_non_habitual_agents'
    values = list(np.arange(0,6000,100))
    measures = ['walking_preparedness', 'maximum_distance']
    simulation_repeats = 5
    
    experiment_simulation(parameter, values, measures, experiment_name = "adding_non_habitual_agents",
        simulation_repeats = simulation_repeats,
        method = 'relMAE')
    
''' 1.10 Adding habitual agents'''
    parameter = 'adding_habitual_agents'
    values = list(np.arange(0,6000,100))
    measures = ['walking_preparedness', 'maximum_distance']
    number_of_agents = 10000
    simulation_repeats = 5
    initialization_repeats = 5
    
    experiment_simulation(parameter, values, measures, experiment_name = "adding_habitual_agents",
        simulation_repeats = simulation_repeats,
        method = 'relMAE', initialization_repeats = initialization_repeats)

IndentationError: unexpected indent (<ipython-input-4-531a55b36f93>, line 2)

In [17]:
import simulation
import data_handler
import environment
import agent
import json
import pandas
import numpy
import datetime
import pickle
#import IPython
import ipywidgets
import bqplot
import multiprocessing
import os
import time
import random
import pandas as pd

imp.reload(data_handler)

<module 'data_handler' from '/home/jurjenvalidation/modeling_charging_behavior/data_handler.py'>

In [5]:
def experiment_initialization_and_simulation(parameter, values, measures, experiment_name = 'Unnamed_experiment',
    number_of_agents = 30, simulation_repeats = 30, method = 'relMAE', initialization_repeats = 1):
    ''' This function runs the experiment that requires initialization as well
        as simulation. For various given values of the specified parameter the
        given measures will be measured. The results are stored in the
        data/experiment_results directory. A subdirectory is created (if it does
        not already exist), containing the measure names and parameter name, in
        which the results are stored.

        Args:
            parameter (str): The parameter to vary for the experiment.
            values (List[int or float]): A list of values the parameter can take.
            measures (List[str]): A list of measure names. Possible measures are
                charging_station_validation, agent_validation,
                time_per_simulation.

        Kwargs:
            number_of_agents (int) Amount of agents to do the experiment with.
                Default is 30.
            simulation_repeats (int): Amount of repeats for the experiment. Default
                is 30.
    '''

    values = list(reversed(values))

    print('INFO: Started experiment: initialization and simulation which varies %s ' %
        parameter + 'with %s as measures. Varying values are %s. ' %
        (measures, [val for val in values]) + '%d agents and %d simulation_repeats.' % \
        (number_of_agents, simulation_repeats))
    start_time = time.process_time()
    print('INFO: Start time: %s' % datetime.datetime.now())

    parameters_file = 'data/input_parameters/parameters.json'
    
    if initialization_repeats > 1:
        parameters = [(parameters_file, parameter, i, len(values), value,
            number_of_agents, simulation_repeats, measures, experiment_name+"_init_repeat_"+str(repeat), method)
            for i, value in enumerate(values) for repeat in range(initialization_repeats)]
    else:
        parameters = [(parameters_file, parameter, i, len(values), value,
            number_of_agents, simulation_repeats, measures, experiment_name, method)
            for i, value in enumerate(values)]
    
    


    inner_loop_experiment_initialization_and_simulation(parameters[0])

    print('INFO: Finished experiment: initialization and simulation which varies %s ' %
        parameter + 'with %s as measures. Varying values are %s. ' %
        (measures, [val for val in values]) + '%d agents and %d simulation_repeats.' % \
        (number_of_agents, simulation_repeats))
    # print('INFO: Experiment took %s' %
    #     get_time_string(time.process_time() - start_time))
    print('INFO: End time: %s' % datetime.datetime.now())

def inner_loop_experiment_initialization_and_simulation(args):
    ''' This function runs an experiment that requires initialization and
        simulation. For a single value of a parameter measuring one or more
        measures are measured. This function is also in charge of storing the
        results of the experiment for this value.

    Args:
        args (Tuple[Any]): Contains the arguments needed for the experiment,
            namely in order:
                - (str): The path to the parameters file;
                - (str): The parameter to vary for the experiment;
                - (int): The current experiment run number;
                - (int): The total amount of values for the parameter;
                - (int or float): The value of the parameter in this run;
                - (int): Amount of agents in the experiment;
                - (int): Amount of repeats for the experiment;
                - (List(str)): A list of measure names. Possible measures are
                    number_of_centers, average_number_of_charging_stations and
                    walking_preparedness;
                - (str): Method of validation, either MAE or relMAE.
    '''

    (parameters_file, parameter, i, max_runs, value, number_of_agents, simulation_repeats, measures, \
     experiment_name, method) = args
    start_time = time.process_time()

    print('INFO: Experiment run %d of %d (%s = %s, measuring %s)' % (i + 1,
        max_runs, parameter, value, measures))

    if "adding_habitual_agents" in experiment_name: # JM: Added for the experiment of adding habitual agents.
        sim = simulation.Simulation(parameters_file, measures = measures,
            overwrite_parameters = {'number_of_agents': 6000,
                                    'agent_initialization': 'load_and_use'})
        
        agent_IDs = list(sim.agents.keys())
        agent_IDs = [agent_ID for agent_ID in agent_IDs if not 'car2go' in agent_ID]
        random.shuffle(agent_IDs)
        
        for i, agent_ID in enumerate(agent_IDs):
            if i > value:
                sim.agents[agent_ID].start_date_agent = sim.stop_condition_parameters['max_time']
            else:
                sim.agents[agent_ID].start_date_agent = sim.start_date_simulation
    else:
        sim = simulation.Simulation(parameters_file, measures = measures,
            overwrite_parameters = {parameter: value,
            'number_of_agents': number_of_agents,
            'agent_initialization': 'create_and_use'}) # JRH: here the create and use is selected as option since the experiment is about the initialization 
                
    print('INFO: Experiment run %d of %d is done with initialization at %s' %
        ((i + 1), max_runs, datetime.datetime.now()))

    newpath = 'data/experiment_results/%s'  % (experiment_name)
    if not os.path.exists(newpath):
        os.makedirs(newpath)
        
    try:
        sim.repeat_simulation(repeat = simulation_repeats, measures = measures, method = method,
            display_progress = False)

        for m in measures:
            experiment_dir = 'data/experiment_results/%s/experiment_initialization_and_simulation_measures_%s_varying_%s' % \
                (experiment_name, m, parameter)
            if not os.path.exists(experiment_dir):
                os.makedirs(experiment_dir)
            
            experiment_filename = experiment_dir + '/%s_%s_%s_agents_%s_simulation_repeats.pkl' % (str(value), parameter, str(number_of_agents), str(simulation_repeats))
            with open(experiment_filename, 'wb') as experiment_file:
                pickle.dump(sim.sensors[m], experiment_file)
            print('INFO: Partial results stored in %s' % experiment_filename)
    except Exception as e:
        print(e)
        print("WARNING: Thus failed")

    print('INFO: Experiment run %d (%s = %s, measuring %s) took %s' %
        (i + 1, parameter, value, measures,
        get_time_string(time.process_time() - start_time)))
    print('INFO: End time: %s' % datetime.datetime.now())

# def main():
#     '''NEW 2.3 Experiment adding habitual users'''
#     parameter = 'adding_habitual_agents'
#     values = list(numpy.arange(1000,6001,1000))
#     measures = ['walking_preparedness', 'maximum_distance']
#     simulation_repeats = 5
#     initialization_repeats = 5
#     number_of_agents = 6000
    
#     experiment_initialization_and_simulation(parameter, values, measures, experiment_name = "adding_habitual_agents",
#         number_of_agents = number_of_agents, simulation_repeats = simulation_repeats,
#         method = 'relMAE', initialization_repeats = initialization_repeats)
    
# if __name__ == '__main__':
#     main()

In [6]:
import pickle
with open('data/agent_database/agents_2014_2019/Agents_2014_2019_all.pkl', 'rb') as pickle_file:
    agents_new = pickle.load(pickle_file)

In [7]:
import imp
import agent
import simulation
imp.reload(simulation)
imp.reload(agent)
parameters_file = 'data/input_parameters/parameters.json'
measures = ['failed_connection_attempt','simulated_sessions'] #'charging_station_validation', 
sim = simulation.Simulation(parameters_file, measures = measures,
            overwrite_parameters = {'number_of_agents': 6000,
                                    'agent_initialization': 'load_and_use',
                                    'add_agents_during_simulation': 0,
                                    "non_habitual_agents": {"Amsterdam":0 ,
                               "Den Haag": 0,
                              "Rotterdam": 0,
                              "Utrecht": 0}
#                                     "filepath_agent_database": "data/agent_database/agents_2014_2019/Individual_Agents/",
#                                     "IDs_from_agent_database": True,
#                                     "start_date_training_data": "01-01-2014",
#         "end_date_training_data": "01-04-2019",
#         "start_date_test_data": "01-01-2014",
#         "end_date_test_data": "01-04-2019",
                                   })


INFO: agent_IDs from parameter file
number_of_agents
agent_initialization
add_agents_during_simulation
non_habitual_agents
INFO: Initialization
	INFO: Loading sessions took 9.12 seconds
	INFO: Detailed preprocessing took 16.87 seconds
	INFO: Started to reset the amount of sockets of the charge points
	INFO: Loading sessions took 7.34 seconds
		INFO: Progress for resetting environment sockets is now at 20 percent.                               
			This took 4.88 minutes
		INFO: Progress for resetting environment sockets is now at 40 percent.                               
			This took 4.55 minutes
		INFO: Progress for resetting environment sockets is now at 60 percent.                               
			This took 4.19 minutes
		INFO: Progress for resetting environment sockets is now at 80 percent.                               
			This took 3.02 minutes
		INFO: Progress for resetting environment sockets is now at 100 percent.                               
			This took 0.32 minutes
	INFO

In [ ]:
type(sim.agents)

In [8]:
#%%pixie_debugger 

sim.repeat_simulation(repeat = 1, 
                      measures = ['time_per_simulation'],
                      display_progress = True)


INFO: Simulation repeat 1 of 1
	INFO: Progress of simulation: 1.0
	INFO: Complete simulation run (1 of 1) took 41.34 minutes


In [9]:
map_figures = ''

for agent in sim.agents:
    agent_object = sim.agents[agent]
    
    dict_agent_validate=agent_object.activity_patterns_test
    
    
    
    

In [11]:
dict_agent_validate

{(4.279766379515419, 52.09401017158591): times
 2000-01-01 00:00:00    0.018538
 2000-01-01 00:20:00    0.018845
 2000-01-01 00:40:00    0.019151
 2000-01-01 01:00:00    0.019151
 2000-01-01 01:20:00    0.019151
                          ...   
 2000-01-01 22:20:00    0.016240
 2000-01-01 22:40:00    0.016700
 2000-01-01 23:00:00    0.016700
 2000-01-01 23:20:00    0.017159
 2000-01-01 23:40:00    0.017772
 Freq: 20T, Name: times, Length: 72, dtype: float64}

In [12]:
df_acticity_training=pd.DataFrame.from_dict( agent.activity_patterns_training)
df_acticity_training.reset_index(inplace=True)
df_acticity_training.columns=['time','density']

AttributeError: 'str' object has no attribute 'activity_patterns_training'

In [13]:
time_value=df_acticity_training.time.dt.time
density=df_acticity_training.density

NameError: name 'df_acticity_training' is not defined

In [18]:
import plotly.graph_objects as go

In [14]:
for agent in sim.agents:
    agent_object = sim.agents[agent]
    
    dict_agent_validate=agent_object.activity_patterns_test
        
    
    
    
    

In [15]:
df_acticity_overall=pd.DataFrame.from_dict( object_agent.overall_activity_pattern)

NameError: name 'object_agent' is not defined

In [153]:
df_acticity_overall.reset_index(inplace=True)

In [141]:

df_acticity_overall.columns=['time','density']

In [160]:
sum(df_acticity_overall.density)

1.0000000000000004

In [159]:
df_acticity_overall.density=df_acticity_overall.density/sum(df_acticity_overall.density)


In [31]:

df_acticity_overall.time.dt.strftime('%H:%M')

0     00:00
1     00:20
2     00:40
3     01:00
4     01:20
      ...  
67    22:20
68    22:40
69    23:00
70    23:20
71    23:40
Name: time, Length: 72, dtype: object

In [34]:
object_agent=sim.agents['car_8263.0']



df_acticity_test=pd.DataFrame.from_dict( object_agent.activity_patterns_test)
df_acticity_test.reset_index(inplace=True)
df_acticity_test.columns=['time','density']



df_acticity_training=pd.DataFrame.from_dict( object_agent.activity_patterns_training)
df_acticity_training.reset_index(inplace=True)
df_acticity_training.columns=['time','density']


df_acticity_overall=pd.DataFrame.from_dict( object_agent.overall_activity_pattern)
df_acticity_overall.columns=['density']
df_acticity_overall.reset_index(inplace=True)
df_acticity_overall.columns=['time','density']
df_acticity_overall.density=df_acticity_overall.density/sum(df_acticity_overall.density)




fig = go.Figure()
fig.add_trace(go.Scatter(x=df_acticity_training.time.dt.strftime('%H:%M'), y=df_acticity_training.density,
                    mode='lines',
                    name='activity pattern training'))


fig.add_trace(go.Scatter(x=df_acticity_test.time.dt.strftime('%H:%M'), y=df_acticity_test.density,
                    mode='lines',
                    name='activity pattern test'))



fig.add_trace(go.Scatter(x=df_acticity_overall.time.dt.strftime('%H:%M'), y=df_acticity_overall.density,
                    mode='lines',
                    name='activity pattern from data'))



fig.update_layout(
                   xaxis_title='time of day',
                   yaxis_title='density')


fig.show()

In [35]:
object_agent.validate()

start get error for<data_handler.DataHandler object at 0x7f7688c3bc88>
expected_pattern length72
simulated_pattern length72
start get error for<data_handler.DataHandler object at 0x7f7688c3bc88>
expected_pattern length72
simulated_pattern length72


{'training': 0.0934518358558834, 'test': 0.10781408062461167}

In [179]:
for agent in sim.agents:

    object_agent=sim.agents[agent]
    dict_agent_validate=object_agent.validate()
    if (dict_agent_validate['test'] <0.05):
        print(agent) 
        print(dict_agent_validate['test'])
    

car_110198.0
0.035089451166468874
car_121749.0
0.04850220752965199
car_159679.0
0.034253105022039265
car_138120.0
0.04325514524294521
car_163381.0
0.04764568652922187
car_2527.0
0.03332587989484519
car_81293.0
0.0487390633352822
car_150836.0
0.03965783895597701
car_84146.0
0.04730854460103117
car_8263.0
0.047984407091089885
car_118975.0
0.04796362952146886
car_168813.0
0.017701024525422715
car_140099.0
0.04582172629887524
car_144282.0
0.04426785756514797
car_11409.0
0.04547777209716378
car_123571.0
0.03478533946083974
car_117420.0
0.04139341843577408
car_173318.0
0.04846118231500129
car_143573.0
0.04118221530450613
car_119119.0
0.044828771478915945
car_155502.0
0.03281126593872666
car_1967.0
0.03954915087781041
car_109225.0
0.03375031462511001
car_155649.0
0.03951440419974342
car_93318.0
0.048722889489609185
car_123660.0
0.038397224210040194
car_50423.0
0.04979340892349114
car_124053.0
0.04133627750517188
car_152451.0
0.04992449569154246
car_160559.0
0.048760668006428136
car_121694.0
0

In [13]:
import pandas

df = pd.DataFrame(sim.sensors['failed_connection_attempt'][0])
print(df.ID.unique())
for agent in list(sim.agents.values()):
    
    print(agent.ID)
    print(agent.simulated_sessions)
all_agents = list(sim.agents.values())
simulated_sessions_of_all_agents_per_run = []
for run in range(len(all_agents[0].all_simulated_sessions)):
    print(run)

    simulated_sessions_of_all_agents_per_run.append(pandas.concat(
                [agent.all_simulated_sessions[0] for agent in all_agents]))


AttributeError: 'DataFrame' object has no attribute 'ID'

In [112]:
df_simulated_sessions_of_all_agents_per_run=simulated_sessions_of_all_agents_per_run[0]

In [117]:
(
    df_simulated_sessions_of_all_agents_per_run
    .query("ID=='car2go_ams_2'")
)

,location_key,ID,start_connection,end_connection,kWh,city,region_abbreviation,provider,address,postal_code,...,longitude,parking_zone,placement_date,status,FirstActiveDateTime,LastActiveDateTime,Chargepoint_IDs,IsFastCharger,IsChargeHub,amount_of_sockets
1,2028,car2go_ams_2,2019-01-08 16:04:00,2019-01-09 00:48:00,32.313333,Amsterdam,Amsterdam,NUON,Julius Pergerstraat 139,1087KN,...,5.002784,ZB08,2014-07-30 17:13:00,1,"[2016-03-01T09:32:03.000000000, 2014-07-30T17:...","[2019-04-30T08:43:45.000000000, 2016-02-29T21:...",{'EVB-P1542193': {'FirstActiveDateTime': 2016-...,0,0,2
2,3156,car2go_ams_2,2019-01-11 18:57:00,2019-01-11 19:14:00,3.116667,Amsterdam,Amsterdam,NUON,Markengouw 1,1024CT,...,4.961597,AN95X,2015-03-04 15:25:47,1,"[2016-10-02T14:30:05.000000000, 2017-01-09T07:...","[2019-03-30T16:34:23.000000000, 2019-04-30T17:...",{'EVB-P1507011': {'FirstActiveDateTime': 2016-...,0,0,2
3,528,car2go_ams_2,2019-01-12 09:34:00,2019-01-12 11:00:00,5.303333,Amsterdam,Amsterdam,NUON,Bijlmerdreef 1285,1103TV,...,4.975480,ZO70,2014-01-02 10:07:00,1,"[2016-03-01T12:09:34.000000000, 2014-01-02T10:...","[2019-04-30T23:55:39.000000000, 2016-02-29T18:...",{'EVB-P1541130': {'FirstActiveDateTime': 2016-...,0,0,2
4,3767,car2go_ams_2,2019-01-15 20:42:00,2019-01-15 23:41:00,32.816667,Amsterdam,Amsterdam,NUON,Brandts Buysstraat 2,1077XC,...,4.871145,BV85I,2014-01-01 11:32:00,1,"[2016-04-01T16:36:16.000000000, 2014-01-01T11:...","[2019-04-29T21:58:56.000000000, 2016-04-01T10:...",{'EVB-P1552383': {'FirstActiveDateTime': 2016-...,0,0,2
5,4606,car2go_ams_2,2019-01-19 23:24:00,2019-01-20 08:00:00,31.820000,Amsterdam,Amsterdam,NUON,Vrolikstraat 315,1091VE,...,4.918053,SO35A,2015-10-22 13:50:03,1,[2015-10-22T13:50:03.000000000],[2019-04-30T15:24:17.000000000],{'EVB-P1512008': {'FirstActiveDateTime': 2015-...,0,0,2
6,9757,car2go_ams_2,2019-01-24 17:42:00,2019-01-24 18:11:00,1.788333,Amsterdam,Amsterdam,NUON,Valeriusstraat 40,1071MN,...,4.870995,OZ30-01,2019-01-09 13:17:55,1,[2015-07-09T10:39:44.000000000],[2019-04-30T14:02:00.000000000],{'EVB-P1523032': {'FirstActiveDateTime': 2015-...,0,0,2
7,1085,car2go_ams_2,2019-01-25 15:31:00,2019-01-25 18:50:00,12.271667,Amsterdam,Amsterdam,NUON,De Boelelaan 30,1083HJ,...,4.886218,BV86,2014-01-02 10:11:00,1,[2014-01-02T10:11:00.000000000],[2019-04-30T19:12:50.000000000],{'EVB-P1525023': {'FirstActiveDateTime': 2014-...,0,0,2
8,755,car2go_ams_2,2019-01-26 16:49:00,2019-01-26 19:07:00,8.510000,Amsterdam,Amsterdam,NUON,Presanella 2,1060NL,...,4.775289,OS76X,2014-06-01 20:51:00,1,"[2016-04-01T19:50:28.000000000, 2014-06-01T20:...","[2019-04-30T08:08:21.000000000, 2016-03-29T20:...",{'EVB-P1552170': {'FirstActiveDateTime': 2016-...,0,0,2
9,816,car2go_ams_2,2019-01-29 21:23:00,2019-01-30 13:22:00,110.000000,Amsterdam,Amsterdam,NUON,Comeniusstraat 249,1065BP,...,4.830252,SV51,2015-05-12 17:40:00,1,"[2016-03-03T14:48:57.000000000, 2015-05-12T17:...","[2019-03-14T07:03:42.000000000, 2016-02-29T17:...",{'EVB-P1542134': {'FirstActiveDateTime': 2016-...,0,0,2
10,5109,car2go_ams_2,2019-02-02 16:21:00,2019-02-02 19:50:00,12.888333,Amsterdam,Amsterdam,NUON,Apollolaan 63,1077AG,...,4.880269,OZ31-03,2016-04-30 11:57:35,1,[2016-02-01T11:10:19.000000000],[2019-04-30T19:21:00.000000000],{'EVB-P1552437': {'FirstActiveDateTime': 2016-...,0,0,2


In [31]:
# agent.training_sessions = agent.training_sessions.sort_values(by='start_connection', inplace = False)
# agent.training_sessions.start_connection = agent.training_sessions.start_connection.shift(-1)
print(len(agent.training_sessions))
import pandas 
pandas.__version__a
agent.training_sessions['disconnection_duration'] = []

0


In [ ]:
method = 'relMAE'
measures = ['failed_connection_attempt', 'simulated_sessions', '']
simulation_repeats = 1
#sim.reset()
sim.repeat_simulation(repeat = simulation_repeats, measures = measures, method = method,
            display_progress = True)

In [54]:
class Simulation():
    def _execute_activity(self, agent):
        ''' This method executes the current activity of the agents and determines
            the time of the next activity of the agent (and possibly the next
            center of the agent).

        Args:
            agent (Agent): The agent whose activity will be executed.

        Updates:
            agent.time_next_activity
            agent.active_center
            agent.active_cs
            environment.css_occupation

        Returns:
            agent.time_next_activity (DateTime): The time of the next activity
                of the agent.
            succesfully_executed_activity (bool): Indicates if the agent has
                succesfully executed their next activity.
        '''


        if agent.is_connected:
            # t = datetime.datetime.now()
            ''' Current activity of agent is disconnecting '''
            self.environment.disconnect_agent(agent.ID, agent.active_cs)
            agent.is_connected = False
            # print('%s: disconnectin1 took %s' % (agent.ID[:5], datetime.datetime.now() - t))
            # t = datetime.datetime.now()

            ''' Next activity of agent is connecting '''
            agent.get_next_connection_time_and_place()
            succesfully_executed_activity = True
            # print('%s: disconnectin2 took %s' % (agent.ID[:5], datetime.datetime.now() - t))

            # print('%s: disconnecting took %s' % (agent.ID[:5], datetime.datetime.now() - t))
        else:
            ''' Current activity of agent is connecting '''
            if agent.is_possible_to_connect(self.current_time):
                # t  = datetime.datetime.now()
                
                # if agent should be deleted, as it is not in use anymore, take its time next activity to be end simulation
                if agent.time_next_activity > agent.end_date_agent:
                    agent.time_next_activity = self.stop_condition_parameters['max_time']
                    succesfully_executed_activity = False
                    return agent.time_next_activity, succesfully_executed_activity

                agent.get_next_disconnection_time()
                agent.select_cs(self.current_time)
                # print('update in _execute_activity')
                print(agent.ID, "update_history")
                agent.update_history(self.current_time)

                agent.is_connected = True
                self.environment.connect_agent(agent.ID, agent.active_cs)

                ''' Next activity of agent is disconnecting '''

                succesfully_executed_activity = True
                agent.sensors['selection_process_attempts'][agent.sensors['run_counter']].append(False)

                # print('%s:    connecting took %s' % (agent.ID[:5], datetime.datetime.now() - t))

            else:
                #agent.history = agent.history[:-1]
                succesfully_executed_activity = False
                agent.sensors['selection_process_attempts'][agent.sensors['run_counter']].append(True)
                
                # if selection_process is strategy model, we are going to simulate the selection process, 
                # even though there are no available cps. This will enable us to measure cascades and 
                # failed cs and where this happens.
                if agent.selection_process == 'strategy_model':
                    agent.select_cs(self.current_time)
                elif agent.selection_process == 'habit_distance':
                    agent.select_cs(self.current_time)
                    
                occupied = set()
                for cs in agent.centers_css[agent.active_center]['distance']:
                    if self.environment.css_info[cs]['placement_date'] < agent.time_next_activity and \
                    self.environment.css_info[cs]['amount_of_sockets'] > 0:
                        # print('3) adding %s' % str(tuple([cs, agent.time_next_activity, tuple(self.environment.who_occupies(cs)), -1])))
                        occupied.add((cs, agent.time_next_activity, tuple(self.environment.who_occupies(cs)), -1))
                        if len(self.environment.who_occupies(cs)) != len(set(self.environment.who_occupies(cs))):
                            print('ERROR: agent twice in sockets of same CP (%s).' % cs)
                            
                #agent.sensors['failed_charge_sessions'][agent.sensors['run_counter']] += 1
                agent.sensors['occupied_counter'][agent.sensors['run_counter']].append(occupied)
                
                
                # if agent should be deleted, as it is not in use anymore, take its time next activity to be end simulation
                if self.current_time < agent.end_date_agent:
                    agent.get_next_connection_time_and_place(failed_session = True)
                else:
                    agent.time_next_activity = self.stop_condition_parameters['max_time']
                
                #agent.time_next_activity = self.current_time + datetime.timedelta(
                #    minutes = agent.time_retry_center)
                
#                 agent._check_next_connection_time(self.current_time, backwards = False,
#                     next_center = agent.active_center)

        return agent.time_next_activity, succesfully_executed_activity

In [51]:
sim._execute_activity = _execute_activity

In [246]:
with open(filepath + filepath2 + '3000_adding_habitual_agents_6000_agents_1_simulation_repeats.pkl', 'rb') as pickle_file:
    simulated_sessions = pickle.load(pickle_file)

In [247]:
df_simulated_sessions= pd.DataFrame.from_records(simulated_sessions[0])


In [248]:
print(df_simulated_sessions)

       location_key            ID    start_connection      end_connection  \
0              6230   car_82594.0 2019-01-09 14:53:00 2019-01-10 08:12:00   
1               611   car_82594.0 2019-01-10 18:25:00 2019-01-11 12:28:00   
2              6230   car_82594.0 2019-01-12 14:57:00 2019-01-13 08:00:00   
3              6230   car_82594.0 2019-01-13 16:44:00 2019-01-14 08:21:00   
4              6005   car_82594.0 2019-01-14 18:31:00 2019-01-15 08:20:00   
...             ...           ...                 ...                 ...   
115744         9476  car_128791.0 2019-03-28 11:12:00 2019-03-28 20:59:00   
115745         9476  car_128791.0 2019-03-29 08:38:00 2019-03-29 11:03:00   
115746         9476  car_128791.0 2019-03-29 11:11:00 2019-03-29 15:41:00   
115747         9476  car_128791.0 2019-03-30 07:50:00 2019-03-30 15:47:00   
115748         9476  car_128791.0 2019-03-31 08:06:00 2019-03-31 14:34:00   

          kWh       city region_abbreviation      provider           addres

In [93]:
filepath2 = 'experiment_initialization_and_simulation_measures_failed_connection_attempt_varying_adding_habitual_agents/'
with open(filepath + filepath2 + '5000_adding_habitual_agents_6000_agents_1_simulation_repeats.pkl', 'rb') as pickle_file:
    failed_connection_attempts = pickle.load(pickle_file)
    
print(failed_connection_attempts)    
    

[   location_key            ID    start_connection      end_connection  \
1          6230   car_82594.0 2019-01-08 18:22:00 2019-01-09 08:18:00   
2          6005   car_82594.0 2019-01-09 18:20:00 2019-01-10 08:13:00   
3          6230   car_82594.0 2019-01-10 18:14:00 2019-01-11 08:29:00   
4          8438   car_82594.0 2019-01-11 18:25:00 2019-01-12 08:08:00   
5          6230   car_82594.0 2019-01-12 18:24:00 2019-01-12 20:43:00   
..          ...           ...                 ...                 ...   
39         2477  car_123852.0 2019-03-17 10:32:00 2019-03-17 15:57:00   
40         2477  car_123852.0 2019-03-23 09:55:00 2019-03-23 17:46:00   
41         5094  car_123852.0 2019-03-24 09:32:00 2019-03-24 16:38:00   
42         2477  car_123852.0 2019-03-25 10:03:00 2019-03-25 17:51:00   
43         5094  car_123852.0 2019-03-26 09:31:00 2019-03-26 16:58:00   

         city region_abbreviation      provider                    address  \
1     Utrecht             Utrecht          N